In [1]:
import numpy as np


In [2]:
from sklearn.metrics import roc_auc_score

In [3]:
import numpy as np
source1 = "F:/MSc DS/Project/dms/trained_dms_txt/bert2dm-pca.txt"
source2 = "F:/MSc DS/Project/dms/trained_dms_txt/bert2dm-pca-cls.txt"
source3 = "F:/MSc DS/Project/dms/trained_dms_txt/bert2dm-svd.txt"
source4 = "F:/MSc DS/Project/dms/trained_dms_txt/bert2dm-svd-cls.txt"
source5 = "F:/MSc DS/Project/dms/trained_dms_txt/ms-word2dm-c5.txt"
source6 = "F:/MSc DS/Project/dms/trained_dms_txt/ms-word2dm-c10.txt"
source7 = "F:/MSc DS/Project/dms/trained_dms_txt/ms-word2dm-d5.txt"
source8 = "F:/MSc DS/Project/dms/trained_dms_txt/ms-word2dm-d10.txt"
source9 = "F:/MSc DS/Project/dms/trained_dms_txt/word2dm.txt"

# dim = 17 #

def get_dict_from_source(source = source5):
    dim =17
    if source ==source1 or source==source3:
        dim = 15
    whole =[]

    with open(source) as f:
        for line in f:
            linelist = line.split()
            for a in linelist:
                whole.append(a)
    chunk = dim**2+1
    n = len(whole)/chunk
    voc = []
    for i in range(int(n)):
        voc.append(whole[i * chunk])
    dic = {}
    for i in range(int(n)):
        voc.append(whole[i * chunk])
    dic = {}
    for i in range(int(n)):
        a = whole[i * chunk + 1:(i + 1) * chunk]
        a = np.array(a, dtype='float')
        a.resize([dim, dim])
        value,_ = np.linalg.eig(a)
        maxeig = max(value)
        dic[voc[i]] = a/maxeig

    return dic

def kba(a, b):
    m = b-a
    eig,_ = np.linalg.eig(m)
    Kba = sum(eig)/sum(abs(eig))
    return Kba

def halfmatrix(m):
    eig,vec = np.linalg.eig(m)
    n = len(eig)
    for i in range(n):
        if eig[i]<0:
            eig[i]=0
    eignew = eig**0.5
    dim = m.shape[0]
    e = np.eye(dim)
    for i in range(dim):
        e[i][i] =eignew[i]
    vec_T = np.linalg.inv(vec)
    half = vec@e@vec_T
    return half

def compute_Kba(listA,listB, dic, method ='add'):
    '''
    imput two phrase or sentences lists
    tokenize each pair and get new dm using composition method
    compute Ke pred for each pair
    if oov set Ke to oov  # after test all in voc

    composition method
    add or pointwise
    '''

    n = len(listA)
    if method == 'add':
        Kba_list = []
        for i in range(n):
            phraseA = listA[i].split()
            phraseB = listB[i].split()
            newA = dic[phraseA[0].lower()] + dic[phraseA[1].lower()]
            newB = dic[phraseB[0].lower()] + dic[phraseB[1].lower()]
            Kba_list.append(kba(newA, newB))

    if method == 'pointwise':
        Kba_list = []
        for i in range(n):
            phraseA = listA[i].split()
            phraseB = listB[i].split()
            newA = dic[phraseA[0].lower()] * dic[phraseA[1].lower()]
            newB = dic[phraseB[0].lower()] * dic[phraseB[1].lower()]
            Kba_list.append(kba(newA, newB))
            
    if method == 'dot':
        Kba_list = []
        for i in range(n):
            phraseA = listA[i].split()
            phraseB = listB[i].split()
            newA = dic[phraseA[0].lower()]@dic[phraseA[1].lower()]
            newB = dic[phraseB[0].lower()]@dic[phraseB[1].lower()]
            Kba_list.append(kba(newA, newB))
    
    if method == 'mult':
        Kba_list = []
        for i in range(n):
            phraseA = listA[i].split()
            phraseB = listB[i].split()
            n1 = dic[phraseA[0].lower()]
            v1 = dic[phraseA[1].lower()]
            n2 = dic[phraseB[0].lower()]
            v2 = dic[phraseB[1].lower()]
            newA = halfmatrix(n1)*v1*halfmatrix(n1)
            newB = halfmatrix(n2)*v2*halfmatrix(n2)
            Kba_list.append(kba(newA, newB))
            
    return Kba_list


# get sv dataset
# get sv dataset
dicA = []
dicB = []
val = []
with open("F:/MSc DS/Project/compositional datasets/KS2016-SV.txt") as f:
    for line in f:
        linelist = line.split(",")
        dicA.append(linelist[0])
        dicB.append(linelist[1])
        val.append(linelist[2])

# convert the T to 1 F to 0 in val list
n = len(val)
for i in range(n):
    if val[i]=='T\n':
        val[i]=1
    else:
        val[i]=0
# dicA  list to store the first phrase
# dicB  list to store the second pharse
# val  true val


# test different source with different composition methods



In [4]:

# source1
dic1 = get_dict_from_source(source1)
# add method
pred = compute_Kba(dicA, dicB,dic1)
from sklearn.metrics import roc_auc_score
result = roc_auc_score(val, pred)
print(result)
pred = compute_Kba(dicA, dicB,dic1,method='pointwise')
from sklearn.metrics import roc_auc_score
result = roc_auc_score(val, pred)
print(result)

0.6348422496570645
0.6115226337448559


In [5]:
dic1 = get_dict_from_source(source2)
# add method
pred = compute_Kba(dicA, dicB,dic1)
result = roc_auc_score(val, pred)
print(result)
pred = compute_Kba(dicA, dicB,dic1,method='pointwise')
result = roc_auc_score(val, pred)
print(result)

0.5168175582990397
0.6076268861454046


In [6]:
dic1 = get_dict_from_source(source3)
# add method
pred = compute_Kba(dicA, dicB,dic1)
result = roc_auc_score(val, pred)
print(result)
pred = compute_Kba(dicA, dicB,dic1,method='pointwise')
result = roc_auc_score(val, pred)
print(result)

0.5276268861454046
0.6480658436213992


In [7]:
dic1 = get_dict_from_source(source4)
# add method
pred = compute_Kba(dicA, dicB,dic1)
result = roc_auc_score(val, pred)
print(result)
pred = compute_Kba(dicA, dicB,dic1,method='pointwise')
result = roc_auc_score(val, pred)
print(result)

0.46573388203017835
0.6568449931412894


In [8]:
dic1 = get_dict_from_source(source5)
# add method
pred = compute_Kba(dicA, dicB,dic1)
result = roc_auc_score(val, pred)
print(result)
pred = compute_Kba(dicA, dicB,dic1,method='pointwise')
result = roc_auc_score(val, pred)
print(result)

0.7653772290809328
0.7468312757201646


In [4]:
dic1 = get_dict_from_source(source6)
# add method
pred = compute_Kba(dicA, dicB,dic1)

result = roc_auc_score(val, pred)


In [5]:
result

0.7964883401920438

In [6]:
pred

[0.0304799659096767,
 -0.0304799659096767,
 0.4309669596934246,
 -0.4309669596934246,
 0.035794062650974276,
 -0.035794062650974276,
 0.1362909078490352,
 -0.1362909078490352,
 0.13636072478030872,
 -0.13636072478030872,
 0.20735811411086869,
 -0.20735811411086869,
 0.1674693821971216,
 -0.1674693821971216,
 -0.21251983809835776,
 0.21251983809835776,
 -0.10103770998700622,
 0.10103770998700622,
 0.01907106351717148,
 -0.01907106351717148,
 0.2748769406069098,
 -0.2748769406069098,
 -0.22087301586063363,
 0.22087301586063363,
 -0.07160990237459544,
 0.07160990237459544,
 -0.06626575809066777,
 0.06626575809066777,
 -0.25505696283063767,
 0.25505696283063767,
 0.1985002092717466,
 -0.1985002092717466,
 0.33244784498971847,
 -0.33244784498971847,
 0.07663318586609823,
 -0.07663318586609823,
 0.17438333249902896,
 -0.17438333249902896,
 -0.009704322831678172,
 0.009704322831678172,
 0.2494845277727186,
 -0.2494845277727186,
 0.23395329775384807,
 -0.23395329775384807,
 0.16610284747885518

In [8]:
dicA

['evidence suggest',
 'information express',
 'people believe',
 'group think',
 'paper present',
 'material show',
 'station serve',
 'facility meet',
 'survey reveal',
 'work show',
 'student develop',
 'person create',
 'company operate',
 'organization manage',
 'player play',
 'contestant compete',
 'study demonstrate',
 'examination show',
 'news come',
 'message travel',
 'Summer finish',
 'season end',
 'report note',
 'document state',
 'book offer',
 'product supply',
 'tree mature',
 'plant grow',
 'title suggest',
 'text state',
 'player choose',
 'person decide',
 'government plan',
 'group intend',
 'News report',
 'information inform',
 'problem persist',
 'condition continue',
 'researcher believe',
 'scientist accept',
 'army invade',
 'force attack',
 'aircraft fly',
 'object travel',
 'movement influence',
 'change affect',
 'man approach',
 'person come',
 'magazine publish',
 'publication produce',
 'son escape',
 'child leave',
 'theatre reopen',
 'building open',

In [ ]:
dicA = []
dicB = []
val = []

In [35]:
import pandas as pd
dic= {'dicA':dicA,'dicB':dicB,'label':val,'pred':pred}
df = pd.DataFrame(dic)

In [36]:
df

,dicA,dicB,label,pred
0,evidence suggest,information express,1,0.030480
1,information express,evidence suggest,0,-0.030480
2,people believe,group think,1,0.430967
3,group think,people believe,0,-0.430967
4,paper present,material show,1,0.035794
...,...,...,...,...
265,unit investigate,council explore,0,-0.058890
266,statistic compare,datum analyze,1,-0.339680
267,datum analyze,statistic compare,0,0.339680
268,board oversee,committee manage,1,0.222617


In [38]:
pred2=[i>0 for i in pred]

In [39]:
pred2 =[int(i==True) for i in pred2]

In [40]:
df['pred2']=pred2

In [41]:
df

,dicA,dicB,label,pred,pred2
0,evidence suggest,information express,1,0.030480,1
1,information express,evidence suggest,0,-0.030480,0
2,people believe,group think,1,0.430967,1
3,group think,people believe,0,-0.430967,0
4,paper present,material show,1,0.035794,1
...,...,...,...,...,...
265,unit investigate,council explore,0,-0.058890,0
266,statistic compare,datum analyze,1,-0.339680,0
267,datum analyze,statistic compare,0,0.339680,1
268,board oversee,committee manage,1,0.222617,1


In [23]:
len(df)

270

In [70]:
df.iloc[[1,3,5]][:]

,dicA,dicB,label,pred,pred2
1,information express,evidence suggest,0,-0.030480,0
3,group think,people believe,0,-0.430967,0
5,material show,paper present,0,-0.035794,0


In [64]:
wrong=[]
for i in range(len(df)):
    if df.label[i]!=df.pred2[i]:
        wrong.append(i)

In [65]:
wrong =[i for i in wrong if i%2==0]

In [66]:
wrong

[14,
 16,
 22,
 24,
 26,
 28,
 38,
 54,
 56,
 58,
 68,
 80,
 86,
 90,
 92,
 94,
 102,
 118,
 124,
 128,
 134,
 136,
 144,
 146,
 164,
 168,
 178,
 200,
 206,
 218,
 234,
 260,
 266]

In [96]:
df_error=df.iloc[common][:]

In [97]:
df_error[['dicA','dicB']]

,dicA,dicB
14,player play,contestant compete
16,study demonstrate,examination show
22,report note,document state
26,tree mature,plant grow
28,title suggest,text state
38,researcher believe,scientist accept
54,range extend,property change
56,mean mean,datum tell
58,study identify,investigation determine
68,public inform,people communicate


In [112]:
kba(dic1['concern'],dic1['state'])

-0.24742300576467607

In [114]:
kba(dic1['report'],dic1['document'])

-0.1764436472370062

In [10]:
dic1 = get_dict_from_source(source7)
# add method
pred = compute_Kba(dicA, dicB,dic1)
result = roc_auc_score(val, pred)
print(result)
pred = compute_Kba(dicA, dicB,dic1,method='pointwise')
result = roc_auc_score(val, pred)
print(result)

0.7698216735253772
0.6050480109739369


In [52]:
dic1 = get_dict_from_source(source8)
# add method
pred = compute_Kba(dicA, dicB,dic1)
result = roc_auc_score(val, pred)
print(result)
pred = compute_Kba(dicA, dicB,dic1,method='pointwise')
result = roc_auc_score(val, pred)
print(result)

0.7512208504801098
0.6658984910836763


In [53]:
pred

[0.025062143813651194,
 -0.025062143813651194,
 0.4030538551090816,
 -0.4030538551090816,
 0.33364533073868713,
 -0.33364533073868713,
 0.3967943399952101,
 -0.3967943399952101,
 0.29268754163525385,
 -0.29268754163525385,
 0.684180105328192,
 -0.684180105328192,
 0.18810798230328327,
 -0.18810798230328327,
 -0.32716240227946164,
 0.32716240227946164,
 -0.559296606796352,
 0.559296606796352,
 -0.387695975824181,
 0.387695975824181,
 0.684851231803545,
 -0.684851231803545,
 -0.5544762960564883,
 0.5544762960564883,
 0.08740355170335941,
 -0.08740355170335941,
 -0.02769329414137613,
 0.02769329414137613,
 -0.7762415394525496,
 0.7762415394525496,
 0.25729218509734614,
 -0.25729218509734614,
 0.6800391336536914,
 -0.6800391336536914,
 0.17505095049281208,
 -0.17505095049281208,
 0.6087455949772508,
 -0.6087455949772508,
 -0.09139056771491309,
 0.09139056771491309,
 0.17722254085899783,
 -0.17722254085899783,
 -0.3258491511147364,
 0.3258491511147364,
 0.24802127682545172,
 -0.248021276825

In [54]:
import pandas as pd
dic2= {'dicA':dicA,'dicB':dicB,'label':val,'pred':pred}
df2 = pd.DataFrame(dic2)

In [55]:
df2

,dicA,dicB,label,pred
0,evidence suggest,information express,1,0.025062
1,information express,evidence suggest,0,-0.025062
2,people believe,group think,1,0.403054
3,group think,people believe,0,-0.403054
4,paper present,material show,1,0.333645
...,...,...,...,...
265,unit investigate,council explore,0,0.572865
266,statistic compare,datum analyze,1,0.013400
267,datum analyze,statistic compare,0,-0.013400
268,board oversee,committee manage,1,0.327685


In [56]:
pred2=[i>0 for i in pred]
pred2 =[int(i==True) for i in pred2]
df2['pred2']=pred2

In [57]:
df2

,dicA,dicB,label,pred,pred2
0,evidence suggest,information express,1,0.025062,1
1,information express,evidence suggest,0,-0.025062,0
2,people believe,group think,1,0.403054,1
3,group think,people believe,0,-0.403054,0
4,paper present,material show,1,0.333645,1
...,...,...,...,...,...
265,unit investigate,council explore,0,0.572865,1
266,statistic compare,datum analyze,1,0.013400,1
267,datum analyze,statistic compare,0,-0.013400,0
268,board oversee,committee manage,1,0.327685,1


In [58]:
wrong2=[]
for i in range(len(df2)):
    if df2.label[i]!=df2.pred2[i]:
        wrong2.append(i)

In [59]:
wrong2

[14,
 15,
 16,
 17,
 18,
 19,
 22,
 23,
 26,
 27,
 28,
 29,
 38,
 39,
 42,
 43,
 54,
 55,
 56,
 57,
 58,
 59,
 62,
 63,
 66,
 67,
 68,
 69,
 70,
 71,
 80,
 81,
 86,
 87,
 90,
 91,
 92,
 93,
 94,
 95,
 100,
 101,
 118,
 119,
 122,
 123,
 124,
 125,
 128,
 129,
 134,
 135,
 140,
 141,
 142,
 143,
 144,
 145,
 154,
 155,
 164,
 165,
 168,
 169,
 172,
 173,
 178,
 179,
 180,
 181,
 192,
 193,
 200,
 201,
 202,
 203,
 206,
 207,
 216,
 217,
 220,
 221,
 224,
 225,
 234,
 235,
 240,
 241,
 252,
 253,
 256,
 257,
 260,
 261,
 262,
 263,
 264,
 265]

In [94]:
common=[]
for i in wrong2:
    if i in wrong:
        common.append(i)

In [95]:
common

[14,
 16,
 22,
 26,
 28,
 38,
 54,
 56,
 58,
 68,
 80,
 86,
 90,
 92,
 94,
 118,
 124,
 128,
 134,
 144,
 164,
 168,
 178,
 200,
 206,
 234,
 260]

In [12]:
dic1 = get_dict_from_source(source9)
# add method
pred = compute_Kba(dicA, dicB,dic1)
result = roc_auc_score(val, pred)
print(result)
pred = compute_Kba(dicA, dicB,dic1,method='pointwise')
result = roc_auc_score(val, pred)
print(result)

0.5582990397805212
0.41893004115226334


In [4]:
dic1 = get_dict_from_source(source1)
pred = compute_Kba(dicA, dicB,dic1,method='mult')
result = roc_auc_score(val, pred)
print(result)

0.6334705075445816


In [5]:
dic1 = get_dict_from_source(source2)
pred = compute_Kba(dicA, dicB,dic1,method='mult')
result = roc_auc_score(val, pred)
print(result)

0.647517146776406


In [6]:
dic1 = get_dict_from_source(source3)
pred = compute_Kba(dicA, dicB,dic1,method='mult')
result = roc_auc_score(val, pred)
print(result)

0.6194238683127571


In [7]:
dic1 = get_dict_from_source(source4)
pred = compute_Kba(dicA, dicB,dic1,method='mult')
result = roc_auc_score(val, pred)
print(result)

0.618161865569273


In [8]:
dic1 = get_dict_from_source(source5)
pred = compute_Kba(dicA, dicB,dic1,method='mult')
result = roc_auc_score(val, pred)
print(result)

0.7297119341563787


In [9]:
dic1 = get_dict_from_source(source6)
pred = compute_Kba(dicA, dicB,dic1,method='mult')
result = roc_auc_score(val, pred)
print(result)

0.7425514403292182


In [10]:
dic1 = get_dict_from_source(source7)
pred = compute_Kba(dicA, dicB,dic1,method='mult')
result = roc_auc_score(val, pred)
print(result)

0.6035116598079561


In [11]:
dic1 = get_dict_from_source(source8)
pred = compute_Kba(dicA, dicB,dic1,method='mult')
result = roc_auc_score(val, pred)
print(result)

0.6774759945130315


In [12]:
dic1 = get_dict_from_source(source9)
pred = compute_Kba(dicA, dicB,dic1,method='mult')
result = roc_auc_score(val, pred)
print(result)

0.43851851851851853
